In [1]:
from powersimdata.scenario.scenario import Scenario
from powersimdata.scenario.scenario_info import ScenarioInfo
import json
import pandas as pd

In [2]:
from powersimdata.scaling.clean_capacity_scaling.auto_capacity_scaling import CollaborativeStrategyManager,\
IndependentStrategyManager, AbstractStrategyManager, TargetManager, ResourceManager, Resource

In [3]:
scenario_string = '394'
targets_info_location ='Eastern Scenario Take 2.csv'

Load a scenario to that we will extract resource information from

In [4]:
scenario = Scenario(scenario_string)

C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\envs\powersimdata\lib\site-packages\paramiko\ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\envs\powersimdata\lib\site-packages\paramiko\kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\envs\powersimdata\lib\site-packages\paramiko\kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed i

SCENARIO: base | EasternBase_2016_Final_2020Feb_take3

--> State
analyze


Create a scenario_info object from which we calculate resource properties 

In [5]:
scenario_info = ScenarioInfo(scenario)

--> Loading PG
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading demand
--> Loading solar
--> Loading wind
--> Loading hydro


Read in external parameters for region capacity targets

In [6]:
eastern = pd.read_csv(targets_info_location)
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,NaN,"solar,wind",NaN,NaN
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,NaN,"solar,wind",NaN,NaN
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
4,Florida,253533010,1.149474,2.914297e+08,0.00,NaN,"solar,wind",NaN,NaN
5,Georgia,171946010,1.028367,1.768236e+08,0.00,NaN,"solar,wind",NaN,NaN
6,Iowa,39669870,1.216553,4.826049e+07,0.00,NaN,"solar,wind",NaN,NaN
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",NaN,NaN
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",NaN,NaN
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",NaN,NaN


Clean up the missing data with appropriate defaults

In [7]:
eastern['external_ce_historical_amount'] = eastern['external_ce_historical_amount'].fillna(0)
eastern['allowed_resources'] = eastern['allowed_resources'].fillna('solar,wind')
eastern['ce_category'] = eastern['ce_category'].fillna('TBD')
eastern['solar_percentage'] = eastern['solar_percentage'].fillna('None')
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,TBD,"solar,wind",0.0,None
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,TBD,"solar,wind",0.0,None
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",0.0,None
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",0.0,None
4,Florida,253533010,1.149474,2.914297e+08,0.00,TBD,"solar,wind",0.0,None
5,Georgia,171946010,1.028367,1.768236e+08,0.00,TBD,"solar,wind",0.0,None
6,Iowa,39669870,1.216553,4.826049e+07,0.00,TBD,"solar,wind",0.0,None
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",0.0,None
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",0.0,None
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",0.0,None


Create Independent and Collaborative Strategy objects and set the simulation hours to be a full year

In [8]:
independent_strategy_manager = IndependentStrategyManager()
collaborative_strategy_manager = CollaborativeStrategyManager()
AbstractStrategyManager.set_next_sim_hours(8784)

Populate strategy managers with targets created from external data

In [9]:
independent_strategy_manager.targets_from_data_frame(eastern)
collaborative_strategy_manager.targets_from_data_frame(eastern)

For each target region, add the available resources and also create allowed resources that are not currently present

In [11]:
# define start and end times of the simulation
start_time = '2016-01-01 00:00:00'
end_time = '2016-12-31 23:00:00'

# add resource objects to targets
independent_strategy_manager.populate_targets_with_resources(scenario_info, int(scenario.info['id']), start_time, end_time)


Alabama

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Arkansas

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Connecticut

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid reso

C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\envs\powersimdata\lib\site-packages\powersimdata\scenario\scenario_info.py:174: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Delaware

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Invalid resource type
Invalid resource type
Added resource other!

No existing resource hydro!
No such type of generator in the area specified. Division by zero.
Added resource hydro!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Florida

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid res

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


North Dakota

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Nebraska

Invalid r

Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


West Virginia

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource ng!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!



In [12]:
collaborative_strategy_manager.populate_targets_with_resources(scenario_info, int(scenario.info['id']), start_time, end_time)


Alabama

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Arkansas

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Connecticut

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid reso

Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Mississippi

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource coal!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Montana Eastern

Invalid resource type
Invalid resource type
Added resource coal!

Invalid resource type
Invalid resource type
Added resource other!

No existing resource hydro!
float division by zero
No such type of generator in the area specified. Division by zero.
Added resource hydro!

Added resource wind!

Invalid re

Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource coal!

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Vermont

Added resource hydro!

Invalid resource type
Invalid resource type
Added resource other!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Added resource wind!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!


Wisconsin

Invalid resource type
Invalid resource type
Added resource ng!

Invalid resource type
Invalid resource type
Added resource nuclear!

Added resource hydro!

Invalid resource type
Invalid r

Calculate the Independent Strategy next capacities along with intermediate values

In [13]:
independent_next_capacities = independent_strategy_manager.data_frame_of_next_capacities()
independent_next_capacities

,ce_target_fraction,ce_target,previous_ce_generation,clean_energy_shortfall,solar_added_capacity,wind_added_capacity,solar_prev_capacity,wind_prev_capacity,solar_expected_cap_factor,wind_expected_cap_factor,next_solar_capacity,next_wind_capacity
region_name,,,,,,,,,,,,
Alabama,0.00,0.000000e+00,1.686780e+05,0.000000e+00,0.000000,0.000000,75.000,1.000,0.2516,0.3334,75.000000,1.000000
Arkansas,0.00,0.000000e+00,3.088913e+04,0.000000e+00,0.000000,0.000000,13.000,1.000,0.2365,0.4416,13.000000,1.000000
Connecticut,0.44,1.369935e+07,5.444679e+05,1.315488e+07,5341.017203,1059.725636,25.200,5.000,0.2073,0.3684,5366.217203,1064.725636
Delaware,0.25,3.607736e+06,6.465725e+04,3.543078e+06,1687.722722,109.595943,30.799,2.000,0.2180,0.3233,1718.521722,111.595943
Florida,0.00,0.000000e+00,6.376034e+05,0.000000e+00,0.000000,0.000000,331.599,3.000,0.2153,0.3984,331.599000,3.000000
Georgia,0.00,0.000000e+00,1.872815e+06,0.000000e+00,0.000000,0.000000,978.100,2.000,0.2172,0.3940,978.100000,2.000000
Iowa,0.00,0.000000e+00,2.284169e+07,0.000000e+00,0.000000,0.000000,2.600,6935.303,0.1919,0.3749,2.600000,6935.303000
Illinois,0.25,4.412802e+07,1.276847e+07,3.135955e+07,84.731376,9960.285106,34.100,4008.500,0.2035,0.3567,118.831376,13968.785106
Indiana,0.10,1.134574e+07,6.363548e+06,4.982189e+06,134.769796,1585.799349,160.597,1889.701,0.1973,0.3409,295.366796,3475.500349


In [ ]:
#independent_next_capacities.to_excel("Eastern_Independent.xlsx")

Calculate the Collaborative Strategy next capacities along with intermediate values

In [14]:
collaborative_next_capacities = collaborative_strategy_manager.data_frame_of_next_capacities()
collaborative_next_capacities

,ce_target_fraction,ce_target,previous_ce_generation,clean_energy_shortfall,solar_scaling,wind_scaling,solar_prev_capacity,wind_prev_capacity,next_solar_capacity,next_wind_capacity
region_name,,,,,,,,,,
Alabama,0.00,0.000000e+00,1.686780e+05,0.000000e+00,3.333214,3.333214,75.000,1.000,249.991082,3.333214
Arkansas,0.00,0.000000e+00,3.088913e+04,0.000000e+00,3.333214,3.333214,13.000,1.000,43.331788,3.333214
Connecticut,0.44,1.369935e+07,5.444679e+05,1.315488e+07,3.333214,3.333214,25.200,5.000,83.997004,16.666072
Delaware,0.25,3.607736e+06,6.465725e+04,3.543078e+06,3.333214,3.333214,30.799,2.000,102.659671,6.666429
Florida,0.00,0.000000e+00,6.376034e+05,0.000000e+00,3.333214,3.333214,331.599,3.000,1105.290571,9.999643
Georgia,0.00,0.000000e+00,1.872815e+06,0.000000e+00,3.333214,3.333214,978.100,2.000,3260.217031,6.666429
Iowa,0.00,0.000000e+00,2.284169e+07,0.000000e+00,3.333214,3.333214,2.600,6935.303,8.666358,23116.852018
Illinois,0.25,4.412802e+07,1.276847e+07,3.135955e+07,3.333214,3.333214,34.100,4008.500,113.662612,13361.190032
Indiana,0.10,1.134574e+07,6.363548e+06,4.982189e+06,3.333214,3.333214,160.597,1889.701,535.304237,6298.778637


Export all target properties as rows: first, use jsonpickle to convert targets to json,
and second, use json_normalize to flatten this json hierarchy into a dataframe

In [15]:
import jsonpickle
from pandas.io.json import json_normalize

cap_planning_df = pd.DataFrame()
for tar in independent_strategy_manager.targets:
    target_df = json_normalize(json.loads(jsonpickle.encode(independent_strategy_manager.targets[tar], unpicklable=False)))
    cap_planning_df = cap_planning_df.append(target_df, sort=False)
cap_planning_df = cap_planning_df.set_index('region_name')
cap_planning_df

,allowed_resources,ce_category,ce_target,ce_target_fraction,external_ce_historical_amount,solar_percentage,total_demand,resources.resources.coal.addl_curtailment,resources.resources.coal.name,resources.resources.coal.no_congestion_cap_factor,...,resources.resources.wind.prev_generation,resources.resources.wind.prev_scenario_num,resources.resources.geothermal.addl_curtailment,resources.resources.geothermal.name,resources.resources.geothermal.no_congestion_cap_factor,resources.resources.geothermal.prev_cap_factor,resources.resources.geothermal.prev_capacity,resources.resources.geothermal.prev_curtailment,resources.resources.geothermal.prev_generation,resources.resources.geothermal.prev_scenario_num
region_name,,,,,,,,,,,,,,,,,,,,,
Alabama,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,8.560028e+07,0.0,coal,0.0,...,2.928641e+03,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arkansas,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,5.764994e+07,0.0,coal,0.0,...,3.879294e+03,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Connecticut,"[geothermal, hydro, solar, wind]",Renewables,1.369935e+07,0.44,0.0,None,3.113488e+07,0.0,coal,0.0,...,1.617796e+04,394,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,394.0
Delaware,"[geothermal, hydro, solar, wind]",Renewables,3.607736e+06,0.25,0.0,None,1.443094e+07,0.0,coal,0.0,...,5.678986e+03,394,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,394.0
Florida,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,2.914297e+08,0.0,coal,0.0,...,1.049981e+04,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Georgia,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,1.768236e+08,0.0,coal,0.0,...,6.921745e+03,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iowa,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,4.826049e+07,0.0,coal,0.0,...,2.283731e+07,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illinois,"[hydro, solar, wind]",Renewables,4.412802e+07,0.25,0.0,None,1.765121e+08,0.0,coal,0.0,...,1.255872e+07,394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Indiana,"[geothermal, hydro, nuclear, solar, wind]",Clean,1.134574e+07,0.10,0.0,None,1.134574e+08,0.0,coal,0.0,...,5.658950e+06,394,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,394.0


In [ ]:
#cap_planning_df.to_excel("Eastern_Capacity_Inputs.xlsx")

In [16]:
unchanged_capacities = cap_planning_df[[
    "resources.resources.coal.prev_capacity",
    "resources.resources.dfo.prev_capacity",
    "resources.resources.geothermal.prev_capacity",
    "resources.resources.hydro.prev_capacity",
    "resources.resources.ng.prev_capacity",
    "resources.resources.nuclear.prev_capacity", 
    "resources.resources.other.prev_capacity"
]].rename(columns={
    "resources.resources.coal.prev_capacity":"Coal",
    "resources.resources.dfo.prev_capacity":"DFO",
    "resources.resources.geothermal.prev_capacity":"Geo-thermal",
    "resources.resources.hydro.prev_capacity":"Hydro",
    "resources.resources.ng.prev_capacity":"Natural Gas",
    "resources.resources.nuclear.prev_capacity":"Nuclear", 
    "resources.resources.other.prev_capacity":"Other"})
unchanged_capacities

,Coal,DFO,Geo-thermal,Hydro,Natural Gas,Nuclear,Other
region_name,,,,,,,
Alabama,6671.201,47.801,NaN,3319.292,15382.996,4547.109,381.000
Arkansas,5487.000,12.300,NaN,1348.902,7229.397,1529.424,156.970
Connecticut,400.000,2816.808,0.0,149.499,3806.601,1888.898,166.000
Delaware,445.500,121.600,0.0,0.000,3016.200,NaN,10.326
Florida,11090.296,5663.306,NaN,55.701,43830.083,3341.230,886.998
Georgia,10109.697,1072.608,NaN,3597.793,18652.301,3929.324,546.008
Iowa,6034.497,1134.119,NaN,129.201,3488.507,535.903,34.476
Illinois,15662.702,808.207,NaN,39.698,17598.997,11236.989,84.349
Indiana,17585.404,285.697,0.0,92.098,8344.201,0.000,91.000


In [17]:
next_capacities = independent_next_capacities[["next_solar_capacity", "next_wind_capacity"]]
next_capacities

,next_solar_capacity,next_wind_capacity
region_name,,
Alabama,75.000000,1.000000
Arkansas,13.000000,1.000000
Connecticut,5366.217203,1064.725636
Delaware,1718.521722,111.595943
Florida,331.599000,3.000000
Georgia,978.100000,2.000000
Iowa,2.600000,6935.303000
Illinois,118.831376,13968.785106
Indiana,295.366796,3475.500349


In [18]:
merged_capacities = pd.concat([unchanged_capacities,next_capacities], axis=1)
merged_capacities = merged_capacities.fillna(0.0)
merged_capacities

,Coal,DFO,Geo-thermal,Hydro,Natural Gas,Nuclear,Other,next_solar_capacity,next_wind_capacity
region_name,,,,,,,,,
Alabama,6671.201,47.801,0.0,3319.292,15382.996,4547.109,381.000,75.000000,1.000000
Arkansas,5487.000,12.300,0.0,1348.902,7229.397,1529.424,156.970,13.000000,1.000000
Connecticut,400.000,2816.808,0.0,149.499,3806.601,1888.898,166.000,5366.217203,1064.725636
Delaware,445.500,121.600,0.0,0.000,3016.200,0.000,10.326,1718.521722,111.595943
Florida,11090.296,5663.306,0.0,55.701,43830.083,3341.230,886.998,331.599000,3.000000
Georgia,10109.697,1072.608,0.0,3597.793,18652.301,3929.324,546.008,978.100000,2.000000
Iowa,6034.497,1134.119,0.0,129.201,3488.507,535.903,34.476,2.600000,6935.303000
Illinois,15662.702,808.207,0.0,39.698,17598.997,11236.989,84.349,118.831376,13968.785106
Indiana,17585.404,285.697,0.0,92.098,8344.201,0.000,91.000,295.366796,3475.500349


In [ ]:
updated_capacities.to_excel("Next_Capacity_Outputs.xlsx")